In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 22: Sporifica Virus ---</h2><p>Diagnostics indicate that the local <em>grid computing cluster</em> has been contaminated with the <em>Sporifica Virus</em>. The grid computing cluster is a seemingly-<span title="The infinite is possible at AdventOfCodeCom.">infinite</span> two-dimensional grid of compute nodes.  Each node is either <em>clean</em> or <em>infected</em> by the virus.</p><p>
</p><p>To <a href="https://en.wikipedia.org/wiki/Morris_worm#The_mistake">prevent overloading</a> the nodes (which would render them useless to the virus) or detection by system administrators, exactly one <em>virus carrier</em> moves through the network, infecting or cleaning nodes as it moves. The virus carrier is always located on a single node in the network (the <em>current node</em>) and keeps track of the <em>direction</em> it is facing.</p>
<p>To avoid detection, the virus carrier works in bursts; in each burst, it <em>wakes up</em>, does some <em>work</em>, and goes back to <em>sleep</em>. The following steps are all executed <em>in order</em> one time each burst:</p>
<ul>
<li>If the <em>current node</em> is <em>infected</em>, it turns to its <em>right</em>.  Otherwise, it turns to its <em>left</em>. (Turning is done in-place; the <em>current node</em> does not change.)</li>
<li>If the <em>current node</em> is <em>clean</em>, it becomes <em>infected</em>.  Otherwise, it becomes <em>cleaned</em>. (This is done <em>after</em> the node is considered for the purposes of changing direction.)</li>
<li>The virus carrier <a href="https://www.youtube.com/watch?v=2vj37yeQQHg">moves</a> <em>forward</em> one node in the direction it is facing.</li>
</ul>
<p>Diagnostics have also provided a <em>map of the node infection status</em> (your puzzle input).  <em>Clean</em> nodes are shown as <code>.</code>; <em>infected</em> nodes are shown as <code>#</code>.  This map only shows the center of the grid; there are many more nodes beyond those shown, but none of them are currently infected.</p>
<p>The virus carrier begins in the middle of the map facing <em>up</em>.</p>
<p>For example, suppose you are given a map like this:</p>
<pre><code>..#
#..
...
</code></pre>
<p>Then, the middle of the infinite grid looks like this, with the virus carrier's position marked with <code>[ ]</code>:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . # . . .
. . . #[.]. . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>The virus carrier is on a <em>clean</em> node, so it turns <em>left</em>, <em>infects</em> the node, and moves left:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . # . . .
. . .[#]# . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>The virus carrier is on an <em>infected</em> node, so it turns <em>right</em>, <em>cleans</em> the node, and moves up:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . .[.]. # . . .
. . . . # . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>Four times in a row, the virus carrier finds a <em>clean</em>, <em>infects</em> it, turns <em>left</em>, and moves forward, ending in the same place and still facing up:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . #[#]. # . . .
. . # # # . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>Now on the same node as before, it sees an infection, which causes it to turn <em>right</em>, <em>clean</em> the node, and move forward:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . # .[.]# . . .
. . # # # . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>After the above actions, a total of <code>7</code> bursts of activity had taken place. Of them, <code>5</code> bursts of activity caused an infection.</p>
<p>After a total of <code>70</code>, the grid looks like this, with the virus carrier facing up:</p>
<pre><code>. . . . . # # . .
. . . . # . . # .
. . . # . . . . #
. . # . #[.]. . #
. . # . # . . # .
. . . . . # # . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>By this time, <code>41</code> bursts of activity caused an infection (though most of those nodes have since been cleaned).</p>
<p>After a total of <code>10000</code> bursts of activity, <code>5587</code> bursts will have caused an infection.</p>
<p>Given your actual map, after <code>10000</code> bursts of activity, <em>how many bursts cause a node to become infected</em>? (Do not count nodes that begin infected.)</p>
</article>


In [2]:
from tabulate import tabulate


example = """
..#
#..
...
"""


def print_grid(infected, p, rows=5, cols=5):
    grid = [
        [
            "X" if (r, c) == p else ("#" if (r, c) in infected else ".")
            for c in range(-cols + 1, cols)
        ]
        for r in range(-rows + 1, rows)
    ]
    print(tabulate(grid, tablefmt="simple_grid"))


def parse(s: str) -> dict[tuple[int, int], int]:
    grid = s.strip().splitlines()
    rows, cols = len(grid), len(grid[0])
    return {
        (r - rows // 2, c - cols // 2): 1
        for r, l in enumerate(grid)
        for c, ch in enumerate(l)
        if ch == "#"
    }


def sporifica_virus(s: str, bursts=1) -> int:
    infected = parse(s)
    directions = ((-1, 0), (0, 1), (1, 0), (0, -1))
    p, d = (0, 0), 0

    infectious_bursts = 0

    for _ in range(bursts):
        if p in infected:
            d = (d + 1) % len(directions)
            del infected[p]
        else:
            d = (d - 1) % len(directions)
            infected[p] = 1
            infectious_bursts += 1

        p = tuple((a + b) for a, b in zip(p, directions[d]))

    return infectious_bursts


print(f"Example 70 bursts: {sporifica_virus(example, 70)} should be 41.")
print(f"Example 10_000: {sporifica_virus(example, 10_000)} should be 5587.")

Example 70 bursts: 41 should be 41.
Example 10_000: 5587 should be 5587.


In [3]:
with open("../input/day22.txt") as f:
    puzzle = f.read()

print(f"Part I: {sporifica_virus(puzzle, 10_000)}")

Part I: 5575


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>5575</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As you go to remove the virus from the infected nodes, it <em>evolves</em> to resist your attempt.</p>
<p>Now, before it infects a clean node, it will <em>weaken</em> it to disable your defenses. If it encounters an infected node, it will instead <em>flag</em> the node to be cleaned in the future.  So:</p>
<ul>
<li><em>Clean</em> nodes become <em>weakened</em>.</li>
<li><em>Weakened</em> nodes become <em>infected</em>.</li>
<li><em>Infected</em> nodes become <em>flagged</em>.</li>
<li><em>Flagged</em> nodes become <em>clean</em>.</li>
</ul>
<p>Every node is always in exactly one of the above states.</p>
<p>The virus carrier still functions in a similar way, but now uses the following logic during its bursts of action:</p>
<ul>
<li>Decide which way to turn based on the <em>current node</em>:
  <ul>
  <li>If it is <em>clean</em>, it turns <em>left</em>.</li>
  <li>If it is <em>weakened</em>, it does <em>not</em> turn, and will continue moving in the same direction.</li>
  <li>If it is <em>infected</em>, it turns <em>right</em>.</li>
  <li>If it is <em>flagged</em>, it <em>reverses</em> direction, and will go back the way it came.</li>
  </ul>
</li>
<li>Modify the state of the <em>current node</em>, as described above.</li>
<li>The virus carrier moves <em>forward</em> one node in the direction it is facing.</li>
</ul>
<p>Start with the same map (still using <code>.</code> for <em>clean</em> and <code>#</code> for infected) and still with the virus carrier starting in the middle and facing <em>up</em>.</p>
<p>Using the same initial state as the previous example, and drawing <em>weakened</em> as <code>W</code> and <em>flagged</em> as <code>F</code>, the middle of the infinite grid looks like this, with the virus carrier's position again marked with <code>[ ]</code>:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . # . . .
. . . #[.]. . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>This is the same as before, since no initial nodes are <em>weakened</em> or <em>flagged</em>.  The virus carrier is on a clean node, so it still turns left, instead <em>weakens</em> the node, and moves left:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . # . . .
. . .[#]W . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>The virus carrier is on an infected node, so it still turns right, instead <em>flags</em> the node, and moves up:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . .[.]. # . . .
. . . F W . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>This process repeats three more times, ending on the previously-flagged node and facing right:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . W W . # . . .
. . W[F]W . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>Finding a flagged node, it reverses direction and <em>cleans</em> the node:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . W W . # . . .
. .[W]. W . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>The <em>weakened</em> node becomes infected, and it continues in the same direction:</p>
<pre><code>. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
. . W W . # . . .
.[.]# . W . . . .
. . . . . . . . .
. . . . . . . . .
. . . . . . . . .
</code></pre>
<p>Of the first <code>100</code> bursts, <code>26</code> will result in <em>infection</em>. Unfortunately, another feature of this evolved virus is <em>speed</em>; of the first <code>10000000</code> bursts, <code>2511944</code> will result in <em>infection</em>.</p>
<p>Given your actual map, after <code>10000000</code> bursts of activity, <em>how many bursts cause a node to become infected</em>? (Do not count nodes that begin infected.)</p>
</article>

</main>


In [4]:
def print_grid_II(infected, p, rows=5, cols=5):
    translate = [".", "W", "#", "F"]
    grid = [
        [
            "X" if (r, c) == p else translate[infected.get((r, c), 0)]
            for c in range(-cols + 1, cols)
        ]
        for r in range(-rows + 1, rows)
    ]
    print(tabulate(grid, tablefmt="simple_grid"))


# Clean nodes become weakened.
# Weakened nodes become infected.
# Infected nodes become flagged.
# Flagged nodes become clean.


def parse_II(s: str) -> dict[tuple[int, int], int]:
    grid = s.strip().splitlines()
    rows, cols = len(grid), len(grid[0])

    return {
        (r - rows // 2, c - cols // 2): 2
        for r, l in enumerate(grid)
        for c, ch in enumerate(l)
        if ch == "#"
    }


def sporifica_virus_evolved(s: str, bursts=1) -> int:
    infected = parse_II(s)
    directions = ((-1, 0), (0, 1), (1, 0), (0, -1))
    p, d = (0, 0), 0

    infectious_bursts = 0

    for _ in range(bursts):
        state = infected.get(p, 0)
        if state == 0:
            d = (d - 1) % len(directions)
        elif state == 2:
            d = (d + 1) % len(directions)
        elif state == 3:
            d = (d + 2) % len(directions)

        infected[p] = (state + 1) % 4
        if infected[p] == 2:
            infectious_bursts += 1
        p = tuple((a + b) for a, b in zip(p, directions[d]))

    return infectious_bursts


print(f"Example 100 bursts: {sporifica_virus_evolved(example, 100)} should be 26.")
print(
    f"Example 10,000,000: {sporifica_virus_evolved(example, 10_000_000)} should be 2511944."
)

Example 100 bursts: 26 should be 26.
Example 10,000,000: 2511944 should be 2511944.


In [5]:
print(f"Part II: {sporifica_virus_evolved(puzzle, 10_000_000)}")

Part II: 2511991


<link href="style.css" rel="stylesheet"></link><main>

<p>Your puzzle answer was <code>2511991</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
